# Lua III
## Metatables (Objects, sort of) & Review

## Metatables 
- Metatables are one of the newest additions to Lua
 - Added in Lua 5.0
- Metatables themselves are ordinary tables
 - They gain their special status through how we attach them to tables
- Metatables can be used to represent user-defined types
- All data types have metatables
 - Only the metatables of tables can be set and modified


## Basic Metatable Use


In [ ]:
Window = {}
Window.prototype = {x = 0, y = 0, width = 100, height = 100}
Window.mt = {}

function Window.new(o)
    setmetatable(o,Window.mt)
    return o
end

Window.mt.__index = Window.prototype

w = Window.new({x = 10, y= 20})
z = Window.new{height = 100, width = 50}

In [ ]:
w.width

In [ ]:
z.x

## What can we do with Metatables
- Set Default values
- Limited Operator Overloading
 - Can only overload a small set of built in functions
 - No concept of method signature
- Reuse a metatable for multiple tables, ie a class
- Fake inheritance

## Setting Default Values
- To set default values when querying, we use the __\_\_index__ metamethod
 - What this actually does is provide a fall back table whose values are accses when the main table returns now for a given key
- As tables can hold functions, this allows a way to do inheritance

## Default Values Example

In [ ]:
Building = {lat = 39.2555, long = 76.7113}
function Building.print(self)
    print("I'm located at " .. self.lat .. ", " .. self.long)
end
Building.__index = Building
UMBC = {}
setmetatable(UMBC,Building)

UMBC.print(UMBC)

## Inheritance Example

In [ ]:
House = {numberOfBathrooms = 35}
function House.printBaths(self)
    print("I have " .. self.numberOfBathrooms .. " bathrooms.")
end

setmetatable(House,Building)
T = {}
T.__index = House
whiteHouse = {lat = 38.8977, long = 77.0336}
setmetatable(whiteHouse,T)--{__index = House})

whiteHouse.print(whiteHouse)
whiteHouse.printBaths(whiteHouse)

## Controlling the Creation of New Table Elements
- Just like the **__index** metamethod is called when an unknown element is accessed, the **__newindex** metamethod is used to update tables
- Defining this will prevent default behavior from occur
    - Causes tables to behave in a read-only like fashion
    - Can be bypassed using the `rawset` function

In [ ]:
T.__newindex = function(t,k,v)
    print("No Updates Allowed")
end
whiteHouse.numberOfBathrooms = 4
whiteHouse.address = "1600 Pennsylvania Avenue"

In [ ]:
T.__newindex = function(t,k,v)
    rawset(t,k,v)
end
whiteHouse.numberOfBathrooms = 4
whiteHouse.address = "1600 Pennsylvania Avenue"
whiteHouse:printBaths()

## Metamethods for Libraries
- A field of a metatable or metamethod does not have to be limited to what Lua expects
- Several libraries can take advantage of metamethods to change their behavior.
    - **__tostring**
    - **__metatable**

In [ ]:
print(whiteHouse)
T.__tostring = function(var)
    return "I'm located at " .. var.lat .. 
    "," .. var.long
end
print(whiteHouse)

In [ ]:
-- The __metatable Method is used to control the behavoir of the 
-- setmetatable and getmetatable functions
print(getmetatable(whiteHouse))
T.__metatable = "T"
print(getmetatable(whiteHouse))
setmetatable(whiteHouse,T)

## Operator Overloading
- To overload a given operator, set the corresponding _metamethod_
 - To overload +, define \_\_add (two underscores)
 - Each metatable only has one metamethod of each name
  - To account for differing operand types, a long sequence of __if__ statements may be needed

## Operator Overloading Example
- Suppose we have a metatable for the class of complex numbers

In [11]:
--Based off Fabio Mascarenhas' example
mt = {}
function mt.__add(c1,c2)
    if(getmetatable(c1) ~= mt) then
        res = {r = c1 + c2.r , i = c2.i}
        setmetatable(res,mt)
        return res
    end
    if(getmetatable(c2) ~= mt) then
        res = {r = c1.r + c2, i = c1.i}
        setmetatable(res,mt)
        return res
    end
    res = {r = c1.r + c2.r , i = c1.i + c2.i}
    setmetatable(res,mt)
    return res
end

In [16]:
complex = {r = 1, i = 2}
complex2 = {r = 2, i = 4}
setmetatable(complex,mt)
setmetatable(complex2,mt)
x = 5 + complex
y = 5
print(string.format("%d + %di",x.r,x.i))
--print(5 - complex)

6 + 2i

## Other Overloadable Operations
- **__mul**
- **__sub**
- **__div**
- **__unm**
- **__pow**
- **__eq**
- **__lt**
- **__le** 

## OOP in Lua
- We will discuss object oriented programming in detail over the next few classes
- In Lua, we can approximate an object by
 - Using metatables to define a class
 - Use the __:__ operator as a shorthand for passing the current value to a function
 

In [ ]:
Animal = {species = 'Dog', noise = "woof"}

function Animal.makeNoise(animal)
    print(animal.species .. " goes " .. animal.noise)
end

Animal.makeNoise(Animal)


In [ ]:
Animal2 = {species = "Dog" , noise = "woof"}

function Animal2:makeNoise(punct)
    print( self.species .. " goes " .. self.noise .. punct )
end

Animal2:makeNoise("!")
Animal2.makeNoise(Animal2,"!")

In [ ]:
Animal3 = {species = "Dog" , noise = "woof"}

function Animal3.makeNoise(self)
    print( self.species .. " goes " .. self.noise )
end

Animal3:makeNoise()

## Object Constructors
- In Lua, the constructor becomes a function that
    - creates a table
    - sets the appropriate metatable
    - and returns the object

In [ ]:
function Animal:new( m_species, m_noise)
    o = {species = m_species, noise = m_noise}
    setmetatable(o, self)
    self.__index = self
    return o
end

function Animal:makeNoise()
    print( self.species .. " goes " .. self.noise )
end

In [ ]:
dog = Animal:new("Dog","Woof")
dog:makeNoise()

cat = Animal:new("Cat","Meow")
cat:makeNoise()

## Issues with OOP in Lua
- Can be cumbersome
- An object is still just a table
- Metatables can be changed
- Consider the output of the following:

In [ ]:
type(Animal3)

In [ ]:
Window = {}
Window.prototype = {x = 0, y = 0, width = 100, height = 100}
Window.mt = {}
Window.mt2 = {}

function Window.new(o)
    setmetatable(o,Window.mt)
    return o
end

Window.mt.__index = Window.prototype
Window.mt2.__index = {color = "red", shape = "circle"}

In [ ]:
x = Window.new{height = 200}
print(x.x,x.y,x.width,x.height)
setmetatable(x,Window.mt2)
print(x.x,x.y,x.width,x.height)

x = {10, 20, 30, 40}
print(x[1])

## Lua Wrap Up and Analysis
- To finish up our discussion of Lua, we are going to run through some things from chapters 5-10 of the textbook

## Names, Binding, and Scope
- Names
 - Case sensitivity?
 - Can reserved words be used?
 - What characters are allowed?
- Binding 
 - When does type binding occur?
- Scope
 - What is the default scope of a variable? 

## Data Types
- What are the data types?
- Can the user define their own data types?
- Arrays 
 - Are subscripts checked?
 - Can we use slicing?
 - How well supported are multidimensional arrays?
- Are pointers accessible?
- Is there type checking?

## Expressions and Assignments
- Does Lua have operator precedence?
- Does Lua allow operator overloading?
- How are type conversions done?
- Does Lua have compound assignment operators (ie +=) ?

## Control Structures
- What is the syntax of an if statement?
- Does Lua have a multiple-select structure (ie switch)?
- What are Lua's counter controlled loops
- What are Lua's logic controlled loops?

## Subprograms
- What is the function syntax in Lua
- Does Lua allow functions to be passed as parameters of other functions?
- Are functions in Lua type-checked
- Can functions in Lua return more than one value?
- Can function definitions be nested?
- Does Lua allow closures?


## Lua Example Application
- A common task in natural language processing is to be able to read a file and calculate various statistics on the words in that file
- As an in-class exercise, we will write together a program that does the following
 - Reads in a text file and breaks it into words, based on spacing.
 - Counts the frequency of each word
 - Prints a the most common words in the file


## Lua Example Application 
### Step 1: Read in the file 

## Lua Example Application 
### Step 2: Count the words

## Lua Example Application 
### Step 3: Print Most Common